# notebook 03: ateco univoco

Authors: Fabio Morea, Leyla Vesnic @ Area Science Park
 
Description: python scripts to clean and prepare data on regional companies.
This script imports data from 'IMPRESE.xlsx' and produces clean data in the form of .csv files.

License: 
scripts are available under CC-BY-4.0
data is not included in the package

Code is mantained on GitLab: https://gitlab.com/area-science-park-sustainability/imprese-fvg

L'assegnazione dell'ateco univoco si basa sulla tipologia degli ateco dichiarati dall'impresa: (I) prevalente, (P) primario e (S) secondario. Laddove presente, il codice univoco corrisponde al codice prevalente, mentre, nei casi in cui il prevalente non è evidente, viene scelto il codice primario (il primo se presenti più di uno).

In [16]:
# Setup
import sys
import os
from pathlib import Path
import datetime
import pandas as pd
#import numpy as np
pd.options.display.max_columns = None

In [17]:
current_path = Path(os.getcwd())
data_subdir = "data"
data_path = current_path.parent / data_subdir
data_dir = str(data_path)
listafile = os.listdir(data_path)
filename = 'imprese_codici.csv'
assert filename in listafile, "Errore: non trovo il file dati"

In [18]:
#crea dizionario con i nomi di colonna originali e corretti
df = pd.read_csv(data_dir + '\\' + filename, sep = '|', dtype='str') 

In [19]:
#elimina le righe senza codici ateco (ateco_tipo = "")
df= df[ df['ateco_tipo'].str.len() >0  ]

In [20]:
#crea info sulle priorità
df['priorita'] = df['ateco_tipo']
df = df.replace({"priorita": { 'I':1, 'P':2, 'S':3} })

In [21]:
df.sort_values(['cf', 'priorita' ,'loc_n'  ], inplace = True)
df

,fonte,mm_aaaa,id_impresa,id_localiz,cf,loc_n,ateco_tipo,ateco,ateco_desc,priorita


In [22]:
def ateco_univoco(dataframe, cf_impresa):
    df_ateco_impresa = dataframe[ dataframe.cf == cf_impresa]
    lista_ateco = []
    for index,row in  df_ateco_impresa.iterrows():
        #print(index, row.ateco_tipo, row.loc_n, row.ateco )
        temp = str(row.ateco_tipo) + str(row.ateco) 
        lista_ateco.append(temp)
    result = lista_ateco[0] if len(lista_ateco)>0 else "n.d."
    return result#, lista_ateco

In [23]:
ateco_univoco(df, '00000470310') #esempio di impresa senza codici, inattiva

'n.d.'

In [24]:
assert ateco_univoco(df, '00000470310') == 'n.d.'

In [25]:
ateco_univoco(df, '00030280267') 

'n.d.'

In [26]:
df2 =  df.drop_duplicates(subset='cf')
df2 = df2.assign(ateco_uni = "") 
df2

,fonte,mm_aaaa,id_impresa,id_localiz,cf,loc_n,ateco_tipo,ateco,ateco_desc,priorita,ateco_uni


In [27]:
from tqdm import tqdm #tiene conto dei tempi e crea una barra di avanzamento

# TODO: questo cicli fa circa 30 iterazioni al secondo... Molto lento: nel caso di 100.000 imprese impiega circa 1 ora. 
# come possiamo renderlo più efficiente

for index,row in tqdm(df2.iterrows()):
    #print('\n\n\n ***', index, row.cf, au)
    df2.loc[index, 'ateco_uni'] = ateco_univoco(df, row.cf)

0it [00:00, ?it/s]


In [28]:
# TODO: si dovrebbe salvare in altra cartella!
df2.to_csv(data_dir + '\\i_ateco_univoco.csv', sep='|', encoding='utf-8-sig', index=False)